# This notebook goes through the procedure of MR and behavioral data processing.
## First - import packages and load data

In [1]:
from neurocov.data_import.import_data import get_all_data
from meteostat import Point, Hourly
from pathlib import Path

data_directory = Path("../data")
qsiprep_dirs = ["/media/groot/Minerva/TheBase/MRI/derivatives/qsiprep", "/media/groot/Yalla/ConnectomePlasticity/MRI/derivatives/qsiprep"]

## Load MR data

In [2]:
import pandas as pd

mr_data = pd.concat([get_all_data(qsiprep_dir) for qsiprep_dir in qsiprep_dirs])
mr_data = mr_data.drop_duplicates(
    subset=[("session_details", "timestamp"), ("questionnaire", "participant"), ("metric", "")], keep="last"
).reset_index(drop=True)

245it [00:29,  8.41it/s]


Removing subjects with less than 2 sessions
Removed 0 subjects
Found a total of 239 subjects


153it [00:50,  3.01it/s]


Removing subjects with less than 2 sessions
Removed 0 subjects
Found a total of 79 subjects


## Load corresponding metorological 

We'll use the [Meteostat](https://dev.meteostat.net/) package to extract [hourly data](https://dev.meteostat.net/python/hourly.html#data-structure) regarding weather and climate.

In [3]:
# Start and end dates of available scans
start_date = mr_data[("session_details", "timestamp")].min()
end_date = mr_data[("session_details", "timestamp")].max()

# Location of the weather station
tel_aviv = {"lon": 34.8, "lat": 32.0833}
point = Point(**tel_aviv)

# Create a hourly object and fetch the data
hourly_met_data = Hourly(point, start_date, end_date).fetch()

# Filter only columns that have (at least some) values
valid_met_columns = hourly_met_data.columns[~hourly_met_data.isnull().all()]
hourly_met_data = hourly_met_data[valid_met_columns]

# Fill missing values with the median of the column
hourly_met_data.fillna(hourly_met_data.median(), inplace=True)

hourly_met_data.reset_index(inplace=True)

## Combine MRI, behavioral and meteorological data.

In [4]:
data = mr_data.copy()
for i, row in data.iterrows():
    # Find the closest date in the hourly data
    closest_datetime = abs(row[("session_details", "timestamp")] - hourly_met_data["time"]).idxmin()
    # Add the hourly meteorological data to the dataframe
    for col in valid_met_columns:
        data.loc[i, ("session_details", col)] = hourly_met_data.loc[closest_datetime, col].astype(float)

## Save for later usage

In [5]:
print(f"Saving combined data to {data_directory.absolute() / 'combined_data.csv'}")
data.to_pickle(data_directory / "data_combined.pickle")

Saving combined data to /home/groot/Projects/PhD/papers/covariates-in-neuroimaging/scripts/../data/combined_data.csv


In [34]:
import os
from pathlib import Path
cmd_template = "rsync -azPL --progress --exclude '*IR*' --exclude '*acq-dwi_dir-*_epi.*' gal@132.66.42.126:/media/hdd/media/MRI/rawdata/{subj}/{ses}/* /media/groot/Minerva/Covid19/MRI/rawdata/{subj}/{ses}/"
origin = Path("/media/groot/Minerva/Covid19/MRI/rawdata_bak")
new = Path("/media/groot/Minerva/Covid19/MRI/rawdata")
# for subj in origin.iterdir():
#     if "182" not in subj.name:
#         continue
#     if not subj.is_dir():
#         continue
#     for ses in subj.iterdir():
#         target = new / subj.name / ses.name
#         if not target.exists():
#             target.mkdir(parents=True)
#         cmd = cmd_template.format(subj=subj.name, ses=ses.name)
#         print(cmd)
#         os.system(cmd)

In [39]:
from connectome_plasticity_project.managers.preprocessing.dmri.qsiprep import Rawdata
from connectome_plasticity_project.managers.preprocessing.dmri.utils import (
    edit_json,
)
test = Path("/media/groot/Minerva/tests/rawdata")

for subj in sorted(new.glob("sub-*")):
    if not subj.is_dir():
        continue
    r = Rawdata(subj)
    r.process()
    fmaps = sorted(list(r.path.glob("ses-*/fmap")))
    if len(fmaps) != len(r.sessions) and len(fmaps) != 0:
        valid_fmap_folder = fmaps[-1]
        valid_fmaps = []
        for fmap in valid_fmap_folder.glob("*.json"):
            valid_fmaps.append(fmap)
        missing_dwi_folders = [ses/"dwi" for ses in r.sessions if not (ses/"fmap").exists()]
        missing_dwis = []
        for missing_dwi_folder in missing_dwi_folders:
            missing_dwis.extend([str(i.relative_to(r.path))for i in missing_dwi_folder.glob("*dwi.nii.gz")])
        for fmap in valid_fmaps:
            edit_json(fmap, missing_dwis)

Renaming /media/groot/Minerva/Covid19/MRI/rawdata/sub-226/ses-202010211656/dwi/sub-226_ses-202010211656_dir-FWD_run-1_sbref.json to /media/groot/Minerva/Covid19/MRI/rawdata/sub-226/ses-202010211656/dwi/sub-226_ses-202010211656_dir-FWD_sbref.json
Renaming /media/groot/Minerva/Covid19/MRI/rawdata/sub-226/ses-202010211656/dwi/sub-226_ses-202010211656_dir-FWD_run-1_sbref.nii.gz to /media/groot/Minerva/Covid19/MRI/rawdata/sub-226/ses-202010211656/dwi/sub-226_ses-202010211656_dir-FWD_sbref.nii.gz
Renaming /media/groot/Minerva/Covid19/MRI/rawdata/sub-226/ses-202010211656/dwi/sub-226_ses-202010211656_dir-FWD_run-1_dwi.json to /media/groot/Minerva/Covid19/MRI/rawdata/sub-226/ses-202010211656/dwi/sub-226_ses-202010211656_dir-FWD_dwi.json
Renaming /media/groot/Minerva/Covid19/MRI/rawdata/sub-226/ses-202010211656/dwi/sub-226_ses-202010211656_dir-FWD_run-1_dwi.bvec to /media/groot/Minerva/Covid19/MRI/rawdata/sub-226/ses-202010211656/dwi/sub-226_ses-202010211656_dir-FWD_dwi.bvec
Renaming /media/groo

In [38]:
r.participant_label

'225'

In [31]:
Path(missing_dwis[0]).relative_to(r.path)

PosixPath('ses-201912251031/dwi/sub-210_ses-201912251031_dir-FWD_dwi.nii.gz')